In [1]:
import json
import os
from sutime import SUTime
jar_files = os.path.join(os.getcwd(), 'jars')
sutime = SUTime(jars=jar_files, mark_time_ranges=True)

/home/ec2-user/anaconda3/envs/bert_tf/lib/python3.7/site-packages/jpype/_core.py:210: UserWarning: 
-------------------------------------------------------------------------------
Deprecated: convertStrings was not specified when starting the JVM. The default
behavior in JPype will be False starting in JPype 0.8. The recommended setting
for new code is convertStrings=False.  The legacy value of True was assumed for
this session. If you are a user of an application that reported this warning,
please file a ticket with the developer.
-------------------------------------------------------------------------------

  """)


In [2]:
test_case = u'I need a desk from 2pm to 3pm tomorrow'
print(json.dumps(sutime.parse(test_case), sort_keys=True, indent=4))

[
    {
        "end": 38,
        "start": 14,
        "text": "from 2pm to 3pm tomorrow",
        "type": "DATE",
        "value": "2019-08-02 INTERSECT (T14:00,T15:00,PT1H)"
    }
]


In [3]:
test_sent = "Give me the objects that were healthy last tuesday and unhealthy tomorrow"
test_sent = " ".join(test_sent.strip().split())

In [4]:
queries = []

In [5]:
import spacy
import en_core_web_sm
from spacy import displacy
from textblob import TextBlob
from textblob.sentiments import NaiveBayesAnalyzer
nlp = spacy.load('../Bert_NER/Data/')
nlp2 = spacy.load('en_core_web_sm')
doc2 = nlp2(test_sent)

In [6]:
print("Before:", [sent.text for sent in doc2.sents])

def set_custom_boundaries(doc):
    for token in doc[:-1]:
        if token.text == "and" or token.text == "but" or token.text == "or" or token.text == "," :
            doc[token.i+1].is_sent_start = True
    return doc

nlp2.add_pipe(set_custom_boundaries, before="parser")
doc2 = nlp2(test_sent)
print("After:", [sent.text for sent in doc2.sents])

Before: ['Give me the objects that were healthy last tuesday and unhealthy tomorrow']
After: ['Give me the objects that were healthy last tuesday and', 'unhealthy tomorrow']


In [7]:
cls = {"ACI_OBJECT":'#E0FFFF',"OPER":'#FFB6C1',"MODF":'#FFFACD',"HEALTH":'#E6E6FA'}

In [19]:
for sent in doc2.sents:
    query = {"object":[],"time":[],"oper":[],"health-score":[]}
    doc = nlp(str(sent))
    h_sent = ""
    for ent in doc.ents:
        if ent.label_=="health":
            h_sent += ent.text + " "
        elif ent.label_=="aci_object":
            query["object"].append(ent.text)
        elif ent.label_=="oper":
            query["oper"].append(ent.text)
    h_sent = h_sent[:-1]
    zen = TextBlob(h_sent, analyzer=NaiveBayesAnalyzer())
    score = 0
    if h_sent:
        if zen.sentiment.classification=='neg':
            query["health-score"].append(0)
#             print('Score is: ',0)
        else:
            query["health-score"].append(100)
            score = 100
#             print('Score is: ',100)
    su_t = sutime.parse(str(sent))
    for it in su_t:
        query['time'].append(it['value'])
        print(json.dumps(su_t, sort_keys=True, indent=4))
#         print(it['value'])
        print(query)
    queries.append(query)
    
    displacy.render(doc, jupyter=True, style='ent',options={'colors':cls})

[
    {
        "end": 50,
        "start": 38,
        "text": "last tuesday",
        "timex-value": "2019-07-23",
        "type": "DATE",
        "value": "2019-07-23"
    }
]
{'object': ['objects'], 'time': ['2019-07-23'], 'oper': [], 'health-score': [100]}


[
    {
        "end": 18,
        "start": 10,
        "text": "tomorrow",
        "timex-value": "2019-08-02",
        "type": "DATE",
        "value": "2019-08-02"
    }
]
{'object': [], 'time': ['2019-08-02'], 'oper': [], 'health-score': [0]}


In [9]:

# for i,sent in enumerate(h_sents):
#     zen = TextBlob(sent, analyzer=NaiveBayesAnalyzer())
#     score = 0
#     if sent:
#         if zen.sentiment.classification=='neg':
#             queries[i]["health-score"].append(0)
#             print('Score is: ',0)
#         else:
#             queries[i]["health-score"].append(100)
#             score = 100
#             print('Score is: ',100)

In [10]:
# import flair
# flair_sentiment = flair.models.TextClassifier.load('en-sentiment')
# s = flair.data.Sentence(sent)
# flair_sentiment.predict(s)
# total_sentiment = s.labels
# total_sentiment

# if sent:
#     if total_sentiment=='neg':
#         query["health-score"].append(0)
#         print('Score is: ',0)
#     else:
#         query["health-score"].append(100)
#         score = 100
#         print('Score is: ',100)

In [11]:
# for i,sent in enumerate(doc2.sents):
#     su_t = sutime.parse(str(sent))
#     for it in su_t:
#         queries[i]['time'].append(it['value'])
#         print(it['value'])
# #     print(json.dumps(su_t, sort_keys=True, indent=4))

In [12]:
# print(test_sent+"\n")
# displacy.render(doc, jupyter=True, style='ent',options={'colors':cls})
# print('\n')
# print(queries)


In [13]:
displacy.render(doc, jupyter=True, style='dep',options={'colors':cls})


In [14]:
import autocomplete
# import pandas as pd
# from autocomplete import models
# data = pd.read_csv("../Bert_NER/Data/training_dataset_2.csv", encoding="latin1").fillna(method="ffill")
# data = " ".join(data['Word'].tolist())
# models.train_models(data)
autocomplete.load()

training!
successfully loaded: models_compressed.pkl
done training!


True

In [15]:
autocomplete.predict('the','v')

[('virtual', 134),
 ('vrf', 58),
 ('vrfs', 47),
 ('vzbrcp', 46),
 ('vpc', 45),
 ('vpcs', 40)]